### check if SVD satisfies CLT

In [41]:
import copy
import os
import sys

from enum import Enum, auto

import numpy as np
import open3d as o3d
import yaml

from tqdm.notebook import tqdm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
sns.set(rc = {'figure.figsize':(20,8)})

dir_module = os.path.dirname(os.path.abspath(""))
print(f"{dir_module=}")
sys.path.append(dir_module)
import augmented_safeguard as asfgd

dir_module='/root/Documents/Projects/augmented_safeguard'


In [42]:
# dataset


dir_dataset = "/mnt/d/"
type_dataset = asfgd.dataset.DatasetType.RIO10
name_train_sequence = "seq01_01"
name_test_sequence = name_train_sequence

dataset_manager = asfgd.dataset.DatasetManager(asfgd.dataset.DatasetType.RIO10, dir_dataset,  name_train_sequence, name_test_sequence)

# NOTE: read camera intrinsics
intrinsics, _ = dataset_manager.read_intrinsics()
print(intrinsics)


# NOTE: read data
NUM_FRAMES = 2
dataset = dataset_manager.read_data(num_frames=NUM_FRAMES)

# NOTE: GT pose
tf = np.loadtxt(dataset["poses"][0])
R = tf[:3, :3]
t = tf[:3, 3]

self.dir_train_dataset='/mnt/d/RIO10/scene01/seq01/seq01_01'
self.dir_test_dataset='/mnt/d/RIO10/scene01/seq01/seq01_01'
PinholeCameraIntrinsic with width = 540 and height = 960.
Access intrinsics with intrinsic_matrix.
[DEBUG] total 2 dataset(s) read! access with keys: dict_keys(['colors', 'depths', 'poses'])


In [43]:
# for each frame, generate samples & accumulate

list_angular_errors_cum = []
sum_angular_errors = 0.0
idx_total_correspondences = 0

for i in tqdm(range(len(dataset["colors"]))):
    pcd_local = dataset_manager.pointcloud_from_rgbd(
        dataset["colors"][i],
        dataset["depths"][i],
        intrinsics 
    )
    pcd_global = copy.deepcopy(pcd_local)
    pcd_global = pcd_global.transform(tf)

    n_kabsch = int(np.asarray(pcd_local.points).shape[0] - np.asarray(pcd_local.points).shape[0]%3)
    # print(f"{n_kabsch=}")

    # strided points
    P = np.asarray(pcd_local.points)
    Q = np.asarray(pcd_global.points)
    P = asfgd.utility.uniform_sample(P, n_kabsch//3)
    Q = asfgd.utility.uniform_sample(Q, n_kabsch//3)
    P = asfgd.utility.blockshaped(P[:n_kabsch], 3, 3)
    Q = asfgd.utility.blockshaped(Q[:n_kabsch], 3, 3)

    for idx, (P_, Q_) in enumerate(zip(P, Q)):
        P_ = np.asmatrix(P_)
        Q_ = np.asmatrix(Q_)

        _, R_, t_ = asfgd.transformations.rigid_transform_3D(P_, Q_, False)

        P_kabsch = (R_ @ Q_.T) + np.tile(t_, (1, 3))
        P_kabsch = P_kabsch.T

        tf_kabsch = asfgd.transformations.pose_from_rot_and_trans(R_, t_.reshape(1, 3))

        err_rot = asfgd.metric.error_angular(R, R_)

        sum_angular_errors += err_rot
        running_avg = sum_angular_errors / (idx_total_correspondences+1)
        if idx_total_correspondences > 100 and running_avg > 10.0:
            print(f"[ERROR] too big rotation error! {running_avg=}")
        list_angular_errors_cum.append(running_avg)

        idx_total_correspondences += 1

    df = pd.DataFrame()
    df["correspondence index"] = [i for i in range(idx_total_correspondences)]
    df["angular error (cum)"] = list_angular_errors_cum

    fig, ax = plt.subplots()
    sns.lineplot(data=df, x="correspondence index", y="angular error (cum)", ax=ax)
    plt.savefig(f"uniform_angular_error_all_cum_{i}.png")
    plt.clf()


  0%|          | 0/2 [00:00<?, ?it/s]

[ERROR] too big rotation error! running_avg=61.32763506929285
[ERROR] too big rotation error! running_avg=61.32763519933716
[ERROR] too big rotation error! running_avg=61.32763532715889
[ERROR] too big rotation error! running_avg=61.327635452757505
[ERROR] too big rotation error! running_avg=61.32763557619492
[ERROR] too big rotation error! running_avg=61.327635697530724
[ERROR] too big rotation error! running_avg=61.32763581688219
[ERROR] too big rotation error! running_avg=61.32763593430875
[ERROR] too big rotation error! running_avg=61.327636049796986
[ERROR] too big rotation error! running_avg=61.32763616345562
[ERROR] too big rotation error! running_avg=61.32763627527566
[ERROR] too big rotation error! running_avg=61.32763638530493
[ERROR] too big rotation error! running_avg=61.32763649365129
[ERROR] too big rotation error! running_avg=61.3276366002964
[ERROR] too big rotation error! running_avg=61.327636705336765
[ERROR] too big rotation error! running_avg=61.32763680875935
[ERRO

KeyboardInterrupt: 